In [ ]:
-- Table: public.user

-- DROP TABLE IF EXISTS public."user";

CREATE TABLE IF NOT EXISTS public."user"
(
    id integer NOT NULL DEFAULT nextval('user_id_seq'::regclass),
    gender character varying(10) COLLATE pg_catalog."default",
    age integer,
    country character varying(50) COLLATE pg_catalog."default",
    city character varying(50) COLLATE pg_catalog."default",
    exp_group integer,
    os character varying(20) COLLATE pg_catalog."default",
    source character varying(50) COLLATE pg_catalog."default",
    CONSTRAINT user_pkey PRIMARY KEY (id)
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public."user"
    OWNER to postgres;

In [ ]:
-- Table: public.post

-- DROP TABLE IF EXISTS public.post;

CREATE TABLE IF NOT EXISTS public.post
(
    id integer NOT NULL DEFAULT nextval('post_id_seq'::regclass),
    text text COLLATE pg_catalog."default",
    topic character varying(100) COLLATE pg_catalog."default",
    CONSTRAINT post_pkey PRIMARY KEY (id)
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public.post
    OWNER to postgres;

In [ ]:
-- Table: public.feed_action

-- DROP TABLE IF EXISTS public.feed_action;

CREATE TABLE IF NOT EXISTS public.feed_action
(
    user_id integer,
    post_id integer,
    action character varying(20) COLLATE pg_catalog."default",
    "time" timestamp without time zone,
    CONSTRAINT feed_action_post_id_fkey FOREIGN KEY (post_id)
        REFERENCES public.post (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION,
    CONSTRAINT feed_action_user_id_fkey FOREIGN KEY (user_id)
        REFERENCES public."user" (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public.feed_action
    OWNER to postgres;

In [ ]:
import os
from dotenv import load_dotenv
from faker import Faker
import random
import psycopg2

# Загрузить переменные окружения
load_dotenv()

# Получить переменную окружения
db_password = os.getenv("DB_PASSWORD")
# Соединение с базой данных
connection = psycopg2.connect(
    database="db-for-task-6", user="postgres", password=db_password, host="localhost", port="5430"
)
cursor = connection.cursor()

fake = Faker()
# Генерация пользователей
for _ in range(100):
    cursor.execute(
        """
        INSERT INTO "user" (gender, age, country, city, exp_group, os, source)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (
            random.choice(['Male', 'Female']),
            random.randint(18, 60),
            fake.country()[:50],
            fake.city()[:50],
            random.randint(0, 3),
            random.choice(['iOS', 'Android', 'Windows']),
            fake.domain_name()[:50]
        )
    )

# Коммитим после вставки всех пользователей
connection.commit()

# Генерация постов
for _ in range(50):
    cursor.execute(
        """
        INSERT INTO "post" (text, topic)
        VALUES (%s, %s)
        """,
        (fake.text(), fake.word())
    )

# Коммитим после вставки всех постов
connection.commit()

# Генерация действий пользователей
for _ in range(200):
    cursor.execute(
        """
        INSERT INTO "feed_action" (user_id, post_id, action, time)
        VALUES (%s, %s, %s, %s)
        """,
        (
            random.randint(1, 100),  # Ограничение диапазона до существующих пользователей
            random.randint(1, 50),   # Ограничение диапазона до существующих постов
            random.choice(['like', 'view']),
            fake.date_time_this_year()
        )
    )

# Коммитим после вставки действий
connection.commit()

# Закрытие соединения
cursor.close()
connection.close()